# Storage Control For Litter Survey

Library Import

In [1]:
# Azure storage system
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# MySQL Database
import mysql.connector

# Panda visualize database
import pandas as pd

# Azure api encrypt data file
import authInfo

import os

Connect blob Storage

In [2]:
# load api access variable
account_name = authInfo.azure_account_name
account_key = authInfo.azure_account_key
container_name = authInfo.azure_container_name

# Create Blob Service Client
blob_service_client = BlobServiceClient(account_url = f"https://{account_name}.blob.core.windows.net", credential = account_key)
# Create a Container Client
container_client = blob_service_client.get_container_client(container_name)

Connection to MySQL Database

In [ ]:
mysql_host = authInfo.sql_host
mysql_user = authInfo.sql_user
mysql_password = authInfo.sql_password
mysql_database = authInfo.sql_database

# print(mysql_database)

db_config = {
    'host' : authInfo.sql_host,
    'user' : authInfo.sql_user,
    'password' : authInfo.sql_password,
    'database' : authInfo.sql_database
}

# Establish a connection
connection = mysql.connector.connect(**db_config)

# Create a cursor object
cursor = connection.cursor()

In [ ]:
# table creation
# table_name = 'litter_survey_annotation_data_sheet'
# create_table_query = f"""
#                     CREATE TABLE {table_name} (
#                         id INT AUTO_INCREMENT PRIMARY KEY,
#                         image_url VARCHAR(255) UNIQUE,
#                         contain_trash ENUM('Y', 'N'),
#                         trash_annotations JSON,
#                         geo_location VARCHAR(255),
#                         lighting_condition VARCHAR(255),
#                         background_type VARCHAR(255),
#                         collection_date DATE,
#                         annotation_date DATE,
#                         annotation_version VARCHAR(10)
#                     );
#                     """

# table update
# insert_table_query = """
#                     INSERT INTO litter_survey_annotation_data_sheet (image_url, contain_trash, trash_annotations, geo_location, lighting_condition, background_type, collection_date, annotation_date, annotation_version)
#                     VALUES 
#                     ('image1.jpg', 'Y', '[{"label": "plastic food wrap", "x": 304, "y": 124},{"label": "bottle", "x": 134, "y": 345}]', 'Location A', 'Daylight', 'Outdoor', '2023-01-01', '2023-01-15', '1.0'),
#                     ('image2.jpg', 'N', '[{"label": "paper cup", "x": 200, "y": 150},{"label": "cigarette butt", "x": 180, "y": 220}]', 'Location B', 'Night', 'Indoor', '2023-01-02', '2023-01-16', '1.1'),
#                     ('image3.jpg', 'Y', '[{"label": "plastic bottle", "x": 50, "y": 100},{"label": "chip bag", "x": 120, "y": 180}]', 'Location C', 'Daylight', 'Outdoor', '2023-01-03', '2023-01-17', '1.2');
#                     """

In [ ]:
# # Execute the CREATE TABLE query
# cursor.execute(insert_table_query)

# # Commit the transaction
# connection.commit()

In [ ]:
# # Show tables in the specified schema
# show_tables_query = f"SHOW TABLES FROM {authInfo.sql_database};"
# cursor.execute(show_tables_query)

# # Fetch the result
# tables = cursor.fetchall()

# # Print the list of tables
# print("Tables in the schema:")
# for table in tables:
#     print(table[0])

In [ ]:
# Define your SQL query
sql_query = "SELECT * FROM litter_survey_annotation_data_sheet"

# Fetch the data into a DataFrame
df = pd.read_sql(sql_query, connection)

# Display the DataFrame
df.head()  # Use df.head() to display the first few rows of the DataFrame

In [ ]:
# Close the cursor and connection
cursor.close()
connection.close()

Get data info from cloud

In [5]:
# Specify the local directory where you want to download the files
local_directory = 'your_local_directory'

# Ensure the local directory exists, or create it if not
os.makedirs(local_directory, exist_ok=True)

In [ ]:
blob_list = container_client.list_blobs()

# Download each blob
for blob in blob_list:
    blob_client = container_client.get_blob_client(blob.name)

    # Specify the local file path for each downloaded file
    local_file_path = os.path.join(local_directory, blob.name)

    # Download the blob to the specified local file path
    with open(local_file_path, 'wb') as local_file:
        blob_data = blob_client.download_blob()
        local_file.write(blob_data.readall())

    print(f"Downloaded: {blob.name} to {local_file_path}")

    # blob_client.delete_blob()
    # print(f"Deleted: {blob.name}")

In [ ]:
blob_list = container_client.list_blobs()
file_amount = 0
for blob in blob_list:
    print(blob.name)
    file_amount += 1
print(file_amount)